In [ ]:
#Step 1 Setup Sample Dataset (Fake Mini Dataset for Testing)

import os
import shutil
import random
from PIL import Image, ImageDraw

# Create folders
categories = ['plastic', 'paper', 'metal']
base_dir = 'recyclables_dataset'

if os.path.exists(base_dir):
    shutil.rmtree(base_dir)

for category in categories:
    os.makedirs(os.path.join(base_dir, category))

# Generate sample images (colored shapes to simulate classes)
def create_sample_image(color, path):
    img = Image.new('RGB', (128, 128), color=color)
    draw = ImageDraw.Draw(img)
    draw.rectangle([30, 30, 90, 90], fill='white')
    img.save(path)

colors = {'plastic': 'blue', 'paper': 'green', 'metal': 'gray'}

# Save 20 fake images per category
for category in categories:
    for i in range(20):
        img_path = os.path.join(base_dir, category, f'{category}_{i}.jpg')
        create_sample_image(colors[category], img_path)


#Step 2: Load Data & Train Model

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

batch_size = 8
img_size = (128, 128)

train_ds = image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
print("Classes:", class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

# Build a lightweight CNN model
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(128, 128, 3)),
    layers.Conv2D(16, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_ds, validation_data=val_ds, epochs=5)


#Step 3: Convert to Tensor Flow Lite

import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# 1. Create dummy data
num_classes = 3
input_shape = (128, 128, 3)
x_train = np.random.random((10, *input_shape)).astype(np.float32)
y_train = np.random.randint(num_classes, size=(10,))

# 2. Build simple model
model = models.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(16, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 3. Train the model
model.fit(x_train, y_train, epochs=1)

# 4. Export model as TensorFlow SavedModel (new way in TF 2.13+)
model.export('saved_model')

# 5. Convert SavedModel to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# 6. Save TFLite model file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ TensorFlow Lite model created successfully!")


#Step 4: Test TFLite Model on One Image

import tensorflow as tf
import numpy as np

# Assume model.tflite was already created and saved properly in Colab's working directory

tflite_model_path = 'model.tflite'  # Make sure this matches the saved filename

# Check if the TFLite model file exists
import os
if not os.path.exists(tflite_model_path):
    raise FileNotFoundError(f"TFLite model file not found at: {tflite_model_path}")

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare a dummy input that matches the model input shape and type
input_shape = input_details[0]['shape']
input_dtype = input_details[0]['dtype']

# Create a dummy input tensor (random or zeros)
dummy_input = np.random.random_sample(input_shape).astype(input_dtype)

# Set the tensor to the interpreter
interpreter.set_tensor(input_details[0]['index'], dummy_input)

# Run inference
interpreter.invoke()

# Get the output
output_data = interpreter.get_tensor(output_details[0]['index'])

print("Output from TFLite model inference:", output_data)




Found 60 files belonging to 3 classes.
Using 48 files for training.
Found 60 files belonging to 3 classes.
Using 12 files for validation.
Classes: ['metal', 'paper', 'plastic']
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 167ms/step - accuracy: 0.2673 - loss: 1.2195 - val_accuracy: 0.7500 - val_loss: 0.5341
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 189ms/step - accuracy: 0.9065 - loss: 0.4062 - val_accuracy: 1.0000 - val_loss: 0.0577
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 205ms/step - accuracy: 1.0000 - loss: 0.0460 - val_accuracy: 1.0000 - val_loss: 0.0044
Epoch 4/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 162ms/step - accuracy: 1.0000 - loss: 0.0036 - val_accuracy: 1.0000 - val_loss: 3.5851e-05
Epoch 5/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step - accuracy: 1.0000 - loss: 3.5336e-05 - val_accuracy: 1.0000 - val_loss: 3.3974e-06
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.1000 - loss: 1.1799
Saved artifact at 'saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor_134')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  140688635948368: T